In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import datetime
from matplotlib.animation import FuncAnimation


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
def csv_to_tensor(csvfile_path):
    # 读取数据文件
    data_df = pd.read_csv(csvfile_path)

    # 提取像素列并转换为适当的格式
    data_df['pixels'] = data_df['pixels'].apply(lambda x: np.array([int(pixel) for pixel in x.split()]).reshape(1, 48, 48))

    # 划分数据集为训练集和测试集
    train_df = data_df[data_df['Usage'] == 'Training']
    test_df = data_df[data_df['Usage'] != 'Training']

    train_features = [torch.tensor(feature, dtype=torch.float32) for feature in train_df['pixels'].values]
    train_features = torch.stack(train_features)/255
    train_labels = torch.tensor(train_df['emotion'].values, dtype=torch.int64)
    test_features = [torch.tensor(feature, dtype=torch.float32) for feature in test_df['pixels'].values]
    test_features = torch.stack(test_features)/255
    test_labels = torch.tensor(test_df['emotion'].values, dtype=torch.int64)

    # public_test_df = data_df[data_df['Usage'] == 'PublicTest']
    # public_test_features = [torch.tensor(feature, dtype=torch.float32) for feature in public_test_df['pixels'].values]
    # public_test_features = torch.stack(public_test_features)/255
    # public_test_labels = torch.tensor(public_test_df['emotion'].values, dtype=torch.int64)

    # private_test_df = data_df[data_df['Usage'] == 'PrivateTest']
    # private_test_features = [torch.tensor(feature, dtype=torch.float32) for feature in private_test_df['pixels'].values]
    # private_test_features = torch.stack(private_test_features)/255
    # private_test_labels = torch.tensor(private_test_df['emotion'].values, dtype=torch.int64)

    return train_features, train_labels, test_features, test_labels
    # return train_features, train_labels, public_test_features, public_test_labels, private_test_features, private_test_labels

# 使用示例
data_file_path = '/content/drive/MyDrive/Colab Notebooks/icml_face_data.csv'
train_features, train_labels, test_features, test_labels = csv_to_tensor(data_file_path)
# train_features, train_labels, public_test_features, public_test_labels, private_test_features, private_test_labels = csv_to_tensor(data_file_path)

NameError: ignored

In [4]:
def add_random_noise(image, mean=0, std=0.1):
    noise = torch.randn_like(image) * std + mean
    noisy_image = image + noise
    noisy_image = torch.clamp(noisy_image, 0, 1)
    return noisy_image
def enlarge_trainset(train_features, train_labels, transform):
    augmented_train_features = torch.stack([transform(train_features[i]) for i in range(train_features.size(0))])
    # return (
    #     torch.cat((train_features, augmented_train_features), dim = 0),
    #     torch.cat((train_labels, train_labels), dim = 0),
    # )
    return (
        augmented_train_features,
        train_labels
    )
transform = transforms.Compose([lambda x: add_random_noise(x),
                                transforms.RandomHorizontalFlip(p=0.5),
                            ])

# transform = transforms.Compose([transforms.RandomErasing(0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
#                                 transforms.RandomHorizontalFlip(p=0.5),
#                             ])
# train_features, train_labels = enlarge_trainset(train_features, train_labels, transform)
# print(train_features.shape)

In [5]:
torch.cuda.is_available()

True

In [29]:
# a=16
# b=32
a=16
b=32
c=120
d=64
# 定义LeNet模型
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, a, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(a, b, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(b * 9 * 9, c)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(c, d)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(d, 7)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, b * 9 * 9)
        x = self.dropout(x)
        x = self.relu3(self.fc1(x))
        x = self.dropout(x)
        x = self.relu4(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [14]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # 输入通道数1，输出通道数64，卷积核大小3x3
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 128, kernel_size=3, padding=1),  # 输入通道数64，输出通道数128，卷积核大小3x3
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 12 * 12, 256),  # 全连接层
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(256, 7)  # 输出层，7个表情类别
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [30]:
# 初始化空的训练和测试准确率列表以及epochs列表
train_accuracy_list = []
test_accuracy_list = []
epochs = []

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [32]:
plt.figure(figsize=(3, 6), dpi=100)
def generateModel(model_path, load_history = False):
    model = LeNet()
    if load_history == True:
        model.load_state_dict(torch.load(model_path))
    return model
def train(model, train_features, train_labels, num_epochs, learning_rate, batch_size):
    global test_features
    print(device)
    model.to(device) # 移动模型到cuda
#     train_features = train_features.to(device) # 移动数据到cuda
#     train_labels = train_labels.to(device)
    dataset = TensorDataset(train_features, train_labels)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    test_accuracy = 0
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()  # 交叉熵损失
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3)
    # 训练模型
    for epoch in range(num_epochs):
        for batch_features, batch_labels in data_loader:
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels.long())  # 计算损失
            loss.backward()
            optimizer.step()
        if epoch % 1 == 0 or epoch == num_epochs - 1:

            model.eval()
            with torch.no_grad():
                predictions = model(test_features)  # 添加通道维度并进行预测
                predicted_labels = torch.argmax(predictions, dim=1)  # 获取预测标签
            test_accuracy = (predicted_labels == test_labels).sum().item() / len(test_labels)
            print(f'Accuracy on testset: {test_accuracy * 100:.2f}%')

            with torch.no_grad():
                predictions = model(train_features)  # 添加通道维度并进行预测
                predicted_labels = torch.argmax(predictions, dim=1)  # 获取预测标签
            accuracy = (predicted_labels == train_labels).sum().item() / len(train_labels)
            print(f'Accuracy on trainset: {accuracy * 100:.2f}%')
            model.train()

            # train_accuracy_list.append(accuracy)
            # test_accuracy_list.append(test_accuracy)
            # epochs.append(epoch + 1)



            #
            # plt.subplot(1, 1, 1)
            # try:
            #     train_acc_lines.remove(train_acc_lines[0])  # 移除上一步曲线
            #     val_acc_lines.remove(val_acc_lines[0])
            # except Exception:
            #     pass
            # train_acc_lines = plt.plot(epochs, train_accuracy_list, 'r', lw=1)  # lw为曲线宽度
            # val_acc_lines = plt.plot(epochs, test_accuracy_list, 'b', lw=1)
            # plt.title("acc")
            # plt.xlabel("epoch")
            # plt.ylabel("acc")
            # plt.legend(["train_acc","test_acc"])
            # plt.show()
            # plt.pause(0.1)  # 图片停留0.1s


        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    model.eval()
    with torch.no_grad():
        predictions = model(test_features)  # 添加通道维度并进行预测
        predicted_labels = torch.argmax(predictions, dim=1)  # 获取预测标签
    test_accuracy = (predicted_labels == test_labels).sum().item() / len(test_labels)
    print(f'Accuracy on testset: {test_accuracy * 100:.2f}%')

    with torch.no_grad():
        predictions = model(train_features)  # 添加通道维度并进行预测
        predicted_labels = torch.argmax(predictions, dim=1)  # 获取预测标签
    accuracy = (predicted_labels == train_labels).sum().item() / len(train_labels)
    print(f'Accuracy on trainset: {accuracy * 100:.2f}%')

    print('Training finished!')
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    model_name = type(model).__name__
    model_file_name = f"{model_name}_{current_time}_{test_accuracy * 100:.2f}"
    # 保存模型
    torch.save(model.state_dict(), model_file_name)
    last_name = model_file_name
    plt.ioff()
test_features = test_features.to(device)
test_labels = test_labels.to(device)
train_features = train_features.to(device)
train_labels = train_labels.to(device)



train(generateModel("LeNet_2023-09-22_06-44-19_54.28",False), train_features, train_labels, 200, 0.001, 512)

cuda:0
Accuracy on testset: 24.71%
Accuracy on trainset: 25.13%
Epoch [1/200], Loss: 1.6715
Accuracy on testset: 33.27%
Accuracy on trainset: 33.02%
Epoch [2/200], Loss: 1.6188
Accuracy on testset: 38.66%
Accuracy on trainset: 37.91%
Epoch [3/200], Loss: 1.4341
Accuracy on testset: 41.11%
Accuracy on trainset: 40.54%
Epoch [4/200], Loss: 1.3891
Accuracy on testset: 41.93%
Accuracy on trainset: 41.63%
Epoch [5/200], Loss: 1.5433
Accuracy on testset: 44.47%
Accuracy on trainset: 44.39%
Epoch [6/200], Loss: 1.5641
Accuracy on testset: 44.80%
Accuracy on trainset: 45.10%
Epoch [7/200], Loss: 1.3449
Accuracy on testset: 47.07%
Accuracy on trainset: 47.90%
Epoch [8/200], Loss: 1.4326
Accuracy on testset: 47.41%
Accuracy on trainset: 48.84%
Epoch [9/200], Loss: 1.2600
Accuracy on testset: 47.59%
Accuracy on trainset: 49.43%
Epoch [10/200], Loss: 1.3670
Accuracy on testset: 49.14%
Accuracy on trainset: 51.09%
Epoch [11/200], Loss: 1.4055
Accuracy on testset: 48.59%
Accuracy on trainset: 50.74%

<Figure size 300x600 with 0 Axes>

In [ ]:
last_name = ""
